In [ ]:
from netCDF4 import Dataset
from datetime import datetime as dt
import numpy as np

In [ ]:
# Enter the start and end time of your analysis period
# Use a format of 'YYYYMMDD_HHMMSS'
strtTime = '20150706_041743'
endTime = '20150706_042336'

# Enter the full file path of your flight-level file (NOAA-HRD netCDF format)
flFile = '/Users/danstechman/GoogleDrive/PECAN-Data/FlightLevelData/20150706I1_AC.nc'

In [ ]:
def timeCalc(strtTime,endTime):
    dtStrt = dt.strptime(strtTime,'%Y%m%d_%H%M%S')
    dtEnd = dt.strptime(endTime,'%Y%m%d_%H%M%S')

    timeDiff = dtEnd - dtStrt
    originT = timeDiff/2
    originTfull = dtStrt+originT

    return timeDiff,originT,originTfull

In [ ]:
def latLonOrigin(originTsecMidnt,flFile):
    flData = Dataset(flFile,'r')
    flLat = flData.variables['LatGPS.3'][:].filled()
    flLon = flData.variables['LonGPS.3'][:].filled()
    flHH = flData.variables['HH'][:]
    flMM = flData.variables['MM'][:]
    flSS = flData.variables['SS'][:]
    flSecs = flHH*3600 + flMM*60 + flSS
    
    matchIx = np.where(flSecs == originTsecMidnt)
    
    latOrigin = flLat[matchIx][0]
    lonOrigin = flLon[matchIx][0]
    
    return latOrigin, lonOrigin

In [ ]:
timeDiff,originT,originTfull = timeCalc(strtTime,endTime)
originTsecMidnt = int((originTfull - originTfull.replace(hour=0, minute=0, second=0)).total_seconds())

latOrigin,lonOrigin = latLonOrigin(originTsecMidnt,flFile)

print( 'Duration of linear track: {} sec'.format(int(timeDiff.total_seconds())))
print( 'Origin time (mid-point) of linear track: {} sec'.format(int(originT.total_seconds())))
print( 'Reference time: {:%H:%M:%S}'.format(originTfull))
print( '\nLatitude at origin time: {:.4f}'.format(latOrigin))
print( 'Longitude at origin time: {:.4f}'.format(lonOrigin))